In [11]:
import sqlite3

import pandas as pd
import random
from datetime import *

columns = 4
rows = 2
df = pd.DataFrame([[random.random() for j in range(columns)] for i in range(rows)], columns=list(["co"+str(i) for i in range(columns)]),index = [datetime(2015,1,1) + timedelta(days=i) for i in range(rows)])

conn = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES|sqlite3.PARSE_COLNAMES)

_createSql = ('''CREATE TABLE IF NOT EXISTS %s( 
                    `id` INTEGER PRIMARY KEY,
        `ts` timestamp DEFAULT NULL,
        `v0` REAL  DEFAULT NULL,
        `v1` REAL DEFAULT NULL,
        `v2` REAL DEFAULT NULL,
        `v3` REAL DEFAULT NULL)'''%("testTable"))
c = conn.cursor()
c.execute(_createSql)
conn.commit()

print(df)
def iterList(df):
    for item in df.iterrows():
        print(type(item[0].to_pydatetime()))
        yield [item[0].to_pydatetime()] + list(item[1])
        
def write(df):
    c.executemany("insert into testTable(ts,v0,v1,v2,v3) values(?,?,?,?,?)",iterList(df))
    conn.commit()
    

write(df)
c.execute("select * from testTable")
c.fetchall()

                 co0       co1       co2       co3
2015-01-01  0.420803  0.167300  0.304596  0.007550
2015-01-02  0.399405  0.329185  0.826581  0.679704
<class 'datetime.datetime'>
<class 'datetime.datetime'>


[(1,
  datetime.datetime(2015, 1, 1, 0, 0),
  0.4208027053004346,
  0.1672996125899885,
  0.3045958686339242,
  0.007550005916727054),
 (2,
  datetime.datetime(2015, 1, 2, 0, 0),
  0.39940480410549084,
  0.32918539198685304,
  0.8265811846665809,
  0.6797044168012963)]

In [8]:
import cProfile
 
#直接把分析结果打印到控制台
# cProfile.run("foo()")
#把分析结果保存到文件中,不过内容可读性差...需要调用pstats模块分析结果
# cProfile.run("foo()", "result")
#还可以直接使用命令行进行操作
#>python -m cProfile myscript.py -o result

import pstats
#创建Stats对象
p = pstats.Stats("result.profile.price")
#这一行的效果和直接运行cProfile.run("foo()")的显示效果是一样的
# p.strip_dirs().sort_stats(-1).print_stats()
#strip_dirs():从所有模块名中去掉无关的路径信息
#sort_stats():把打印信息按照标准的module/name/line字符串进行排序
#print_stats():打印出所有分析信息

#按照函数名排序 
# p.strip_dirs().sort_stats("name").print_stats()

#按照在一个函数中累积的运行时间进行排序
#print_stats(3):只打印前3行函数的信息,参数还可为小数,表示前百分之几的函数信息
p.strip_dirs().sort_stats("cumulative").print_stats(30)

#还有一种用法
# p.sort_stats('time', 'cum').print_stats(.5, 'foo')
#先按time排序,再按cumulative时间排序,然后打倒出前50%中含有函数信息

#如果想知道有哪些函数调用了bar,可使用
# p.print_callers(0.5, "bar")

#同理,查看foo()函数中调用了哪些函数
# p.print_callees("foo")

Thu Jan 25 18:34:06 2018    result.profile.price

         13548402 function calls (13495536 primitive calls) in 101.302 seconds

   Ordered by: cumulative time
   List reduced from 5540 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    901/1    0.041    0.000  101.409  101.409 {built-in method builtins.exec}
        1    0.000    0.000  101.409  101.409 test.py:59(test)
        1    0.000    0.000  101.409  101.409 test.py:36(evaluate)
        1    0.002    0.002   98.939   98.939 evaluate.py:54(evaluateRun)
        1    0.071    0.071   50.629   50.629 evaluate.py:256(Prices)
     3547    0.015    0.000   49.901    0.014 data_proxy.py:148(history_bars)
     3547    0.094    0.000   49.874    0.014 base_data_source.py:155(history_bars)
     3547    0.017    0.000   49.517    0.014 base_data_source.py:109(_all_day_bars_of)
     3547    1.095    0.000   49.496    0.014 daybar_store.py:37(get_bars)
    31930    0.597    0.000   45.